#📌 Extracción

In [1]:
import pandas as pd

In [2]:
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

In [3]:
df_raw = pd.read_json(url)
df_raw.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformación

In [5]:
df_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


In [6]:
df_raw.columns


Index(['customerID', 'Churn', 'customer', 'phone', 'internet', 'account'], dtype='object')

In [7]:
df_raw.describe(include='all')


,customerID,Churn,customer,phone,internet,account
count,7267,7267,7267,7267,7267,7267
unique,7267,3,891,3,129,6931
top,9995-HOTOH,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'No', 'OnlineSecurity': 'N...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
freq,1,5174,223,3495,1581,6


In [12]:
df_raw.loc[0, 'account']


{'Contract': 'One year',
 'PaperlessBilling': 'Yes',
 'PaymentMethod': 'Mailed check',
 'Charges': {'Monthly': 65.6, 'Total': '593.3'}}

In [13]:
df_customer = pd.json_normalize(df_raw['customer'])
df_phone = pd.json_normalize(df_raw['phone'])
df_internet = pd.json_normalize(df_raw['internet'])
df_account = pd.json_normalize(df_raw['account'])


In [15]:
df = pd.concat([
    df_raw[['customerid', 'churn']],
    df_customer,
    df_phone,
    df_internet,
    df_account
], axis=1)


In [16]:
df.head()
df.columns


Index(['customerid', 'churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly',
       'Charges.Total'],
      dtype='object')

In [19]:
df['Charges.Total'] = pd.to_numeric(df['Charges.Total'], errors='coerce')
df['Charges.Monthly'] = pd.to_numeric(df['Charges.Monthly'], errors='coerce')


In [20]:
df.isna().sum()


,0
customerid,0
churn,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0


In [21]:
cols_churn = [
    'churn',
    'tenure',
    'Contract',
    'InternetService',
    'Charges.Monthly',
    'Charges.Total',
    'PaymentMethod',
    'TechSupport',
    'OnlineSecurity'
]

df_churn = df[cols_churn]
df_churn.head()


,churn,tenure,Contract,InternetService,Charges.Monthly,Charges.Total,PaymentMethod,TechSupport,OnlineSecurity
0,No,9,One year,DSL,65.6,593.30,Mailed check,Yes,No
1,No,9,Month-to-month,DSL,59.9,542.40,Mailed check,No,No
2,Yes,4,Month-to-month,Fiber optic,73.9,280.85,Electronic check,No,No
3,Yes,13,Month-to-month,Fiber optic,98.0,1237.85,Electronic check,No,No
4,Yes,3,Month-to-month,Fiber optic,83.9,267.40,Mailed check,Yes,No


### Comprensión del Dataset

Tras el proceso de extracción y transformación, el dataset final cuenta con 21 columnas que integran información demográfica, servicios contratados, datos contractuales y financieros de los clientes de Telecom X.  
La estructura final permite realizar análisis exploratorios y modelado predictivo de la evasión de clientes (churn).

Se identificaron como variables clave aquellas relacionadas con la antigüedad del cliente, tipo de contrato, servicios de internet y cargos financieros.


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#📊 Carga y análisis

#📄Informe final